In [75]:
import os 
import cellpy 
from cellpy import cellreader
from cellpy import prmreader

In [76]:
file_name  = r"C:\users\thomasvandijk\Cellpyleafs\534_ES_Fe02CDvsNi_Retsch_Wet.res"
mass      = 0.982 # mass of active material in mg
out_folder = r"C:\users\thomasvandijk\Cellpyleafs"

In [77]:
d = cellreader.CellpyData()
d.from_raw(file_name) # this tells cellpy to read the arbin data file (.res format)
d.set_mass(mass)


In [78]:
d.make_step_table()
d.make_summary()
list_of_cycles = d.get_cycle_numbers()
number_of_cycles = len(list_of_cycles)
print(f"you have {number_of_cycles} cycles")
current,voltage = d.get_cap(5) # current and voltage for cycle 5

you have 367 cycles


In [79]:
c = d.cell

In [80]:
import plotly.express as px
df_raw= c.raw
df_raw_small = c.raw.iloc[0:3,0:12] 
fig = px.scatter(df_raw, y="voltage", title='Discharge Capacity')
#fig.show()

In [81]:
df_sum = c.summary
df_sum_small = df_sum.iloc[0:3,0:10]

In [82]:
df_sum['cycle_index']=df_sum.index

In [83]:
df_sum2 = df_sum[['cycle_index','data_point', 'test_time', 'date_time', 'end_voltage_charge_u_V','end_voltage_discharge_u_V', 'charge_capacity', 'discharge_capacity','discharge_capacity_u_mAh_g', 'charge_capacity_u_mAh_g','cumulated_discharge_capacity_u_mAh_g','cumulated_charge_capacity_u_mAh_g', 'coulombic_efficiency_u_percentage', 'coulombic_difference_u_mAh_g','cumulated_coulombic_efficiency_u_percentage','cumulated_coulombic_difference_u_mAh_g','discharge_capacity_loss_u_mAh_g','cumulated_discharge_capacity_loss_u_mAh_g','charge_capacity_loss_u_mAh_g','cumulated_charge_capacity_loss_u_mAh_g', 'low_level_u_percentage','high_level_u_percentage', 'cumulated_ric_u_none','cumulated_ric_sei_u_none', 'cumulated_ric_disconnect_u_none','shifted_charge_capacity_u_mAh_g', 'shifted_discharge_capacity_u_mAh_g','normalized_cycle_index', 'charge_c_rate', 'discharge_c_rate']]
       

In [84]:
import json 
out_summary = json.loads(df_sum2.to_json(orient='split')) # change  df_sum_small when you want to see the structure
out_raw = json.loads(df_raw.to_json(orient='split')) # change  df_raw_small when you want to see the structure

In [85]:
# the json structure, four arrays in 1 json object. 
# The experiment_info array might become bigger. We might want to read-out more data from the .res file. The auxiliary table, if existing, will be as long as the experiment_data file
xx={
        "experiment_info":{
            "channel_number":c.channel_number,
            "schedule_file_name" : c.schedule_file_name},
        "experiment_summary": out_summary,
        "auxiliary_table":{
            "columns":["test_time","date_time"],
            "index":[1,2,3],
            "data":[[12029.0603495076,1572450446000],[20490.1864950906,1572458908000],[29256.6799169249,1572467675000]]},
         "experiment_data": out_raw
    }

In [86]:
with open("charge-discharge3.json", "w") as f: 
    json.dump(xx,f)

In [87]:
import gzip
import shutil 
with open("charge-discharge3.json",'rb') as f_in: 
    with gzip.open('charge-discharge3.json.gz','wb') as f_out:
        shutil.copyfileobj(f_in,f_out)